# Implement k-Nearest Neighbors in Python

    1. Xem xét dữ liệu
    1.1. Training set
    1.2. Test set
    2. Import thư viện
    3. Load data (tải dữ liệu)
    4. Biến đổi giá trị attribute thành float number
    5. Nhận nhãn dữ liệu huấn luyện
    6. Tìm k-neighbor
    7. Gán nhãn có dữ liệu thử nhiệm với maximum vote
    
# Đặt vấn đề

    Chúng ta có:
    - Một tập dữ liệu huấn luyện (training set) và 
    - Một tập dữ liệu cần được phân lớp là nó thuộc trong class nào (test set)
(Class và label trong bài này mình coi như là có ý nghĩa giống nhau)

# Cách thực hiện

    Trong bài toán classification, nhãn của dữ liệu mới được suy ra từ K điểm dữ liệu trong training set gần nhất. Nhãn đó được quyết định bằng bầu chọn theo số phiếu giữa các điểm gần nhất.
    
# Algorithm
```
    Input: Training set, Test set
    Output: Label for test data point in Test set
    
    Foreach x in Test_set do
        Compute distances(x, yi) // yi in Training_set
        sort the computed distances
        select k-nearest neighbor corresponding to k smallest distances;
```

# 1. Xem xet dữ liệu

Trong cùng thư mục này có hai tệp:
    - irist.csv : là training set
    - iris-test.csv : là test set
    
Trong đó training set và test set gồm có attributes và nhãn.

* Attributes:
    * sepal-length
    * sepal-width
    * petal-length
    * petal-width
* Nhãn (class):
    * Iris-setosa
    * Iris-versicolor
    * Iris-virginica
    
Tổng: 150 hàng, 5 cột.

In [1]:
# Iris dataset
import pandas
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
training_data = pandas.read_csv('iris.csv', names=names)
test_data = pandas.read_csv('iris-test.csv', names=names)

# 1.1 Training set

In [2]:
print(training_data.shape)

(150, 5)


In [3]:
# head
print(training_data.head(10))

   sepal-length  sepal-width  petal-length  petal-width   class
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
5           5.4          3.9           1.7          0.4  setosa
6           4.6          3.4           1.4          0.3  setosa
7           5.0          3.4           1.5          0.2  setosa
8           4.4          2.9           1.4          0.2  setosa
9           4.9          3.1           1.5          0.1  setosa


In [4]:
# class distribution
print(training_data.groupby('class').size())

class
setosa        50
versicolor    50
virginica     50
dtype: int64


In [5]:
# statistical summary
print(training_data.describe())

       sepal-length  sepal-width  petal-length  petal-width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.054000      3.758667     1.198667
std        0.828066     0.433594      1.764420     0.763161
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000


# 1.2. Test set

In [6]:
# Test data
print(test_data.shape)

(12, 5)


In [7]:
print(test_data)

    sepal-length  sepal-width  petal-length  petal-width  class
0            4.3          2.9           1.7          0.3    NaN
1            4.6          2.7           1.5          0.2    NaN
2            5.3          3.4           1.6          0.2    NaN
3            5.2          4.1           1.5          0.1    NaN
4            6.0          2.2           4.2          1.0    NaN
5            6.2          2.3           4.5          1.5    NaN
6            5.0          2.1           3.6          1.2    NaN
7            6.6          2.8           5.4          2.0    NaN
8            6.4          3.2           5.3          2.3    NaN
9            7.0          3.1           5.5          1.8    NaN
10           6.2          3.3           5.9          2.1    NaN
11           6.6          2.9           5.3          2.3    NaN


In [8]:
print(test_data.describe())

       sepal-length  sepal-width  petal-length  petal-width  class
count     12.000000    12.000000     12.000000    12.000000    0.0
mean       5.783333     2.916667      3.833333     1.250000    NaN
std        0.872648     0.565418      1.780364     0.871258    NaN
min        4.300000     2.100000      1.500000     0.100000    NaN
25%        5.150000     2.600000      1.675000     0.275000    NaN
50%        6.100000     2.900000      4.350000     1.350000    NaN
75%        6.450000     3.225000      5.325000     2.025000    NaN
max        7.000000     4.100000      5.900000     2.300000    NaN


# Implement KNN in Python

# 2. Import thư viện

In [9]:
# Import library
from csv import reader
from sys import exit
from math import sqrt
from operator import itemgetter

# 3. Load data

In [10]:
# 1. Load dataset
def load_data_set(filename):
    try:
        with open(filename, newline='') as iris:
            return list(reader(iris, delimiter=','))
    except FileNotFoundError as e:
        raise e

# 4. Biến đổi giá trị của attribute thành float number

In [11]:
# Convert
def convert_to_float(data_set, mode):
    new_set = []
    try:
        if mode == 'training':
            for data in data_set:
                new_set.append([float(x) for x in data[:len(data)-1]] + [data[len(data)-1]])

        elif mode == 'test':
            for data in data_set:
                new_set.append([float(x) for x in data])

        else:
            print('Invalid mode, program will exit.')
            exit()

        return new_set

    except ValueError as v:
        print(v)
        print('Invalid data set format, program will exit.')
        exit()
        

# 5. Nhận nhãn dữ liệu

In [12]:
# get class
def get_classes(training_set):
    return list(set([c[-1] for c in training_set]))

# 6. Tìm số k-Neighbors ứng với khoảng cách của nó

In [13]:
# Find K-Nearest Neightbors
def find_neighbors(distances, k):
    return distances[0:k]

# 7. Gán nhãn cho dữ liệu dựa vào maximum vote

In [14]:
# Find response
def find_response(neighbors, classes):
    votes = [0] * len(classes)

    for instance in neighbors:
        for ctr, c in enumerate(classes):
            if instance[-2] == c:
                votes[ctr] += 1

    return max(enumerate(votes), key=itemgetter(1))

# 8. Thuật toán KNN

In [15]:
# KNN
def knn(training_set, test_set, k):
    distances = []
    dist = 0
    limit = len(training_set[0]) - 1

    # generate response classes from training data
    classes = get_classes(training_set)

    try:
        for test_instance in test_set:
            for row in training_set:
                for x, y in zip(row[:limit], test_instance):
                    dist += (x-y) * (x-y)
                distances.append(row + [sqrt(dist)])
                dist = 0

            distances.sort(key=itemgetter(len(distances[0])-1))

            # find k nearest neighbors
            neighbors = find_neighbors(distances, k)

            # get the class with maximum votes
            index, value = find_response(neighbors, classes)

            # Display prediction
            print('The predicted class for sample ' + str(test_instance) + ' is : ' + classes[index])
            print('Number of votes : ' + str(value) + ' out of ' + str(k))

            # empty the distance list
            distances.clear()

    except Exception as e:
        print(e)

# 9. Chương trình chính

In [16]:
# Main
def main():
    try:
        # get value of k
        k = int(input('Enter the value of k : '))

        # load the training and test data set
        training_file = input('Enter name of training data file : ')
        test_file = input('Enter name of test data file : ')
        training_set = convert_to_float(load_data_set(training_file), 'training')
        test_set = convert_to_float(load_data_set(test_file), 'test')

        if not training_set:
            print('Empty training set')

        elif not test_set:
            print('Empty test set')

        elif k > len(training_set):
            print('Expected number of neighbors is higher than number of training data instances')

        else:
            knn(training_set, test_set, k)
            print('Accuracy: ')
        
        

    except ValueError as v:
        print(v)

    except FileNotFoundError:
        print('File not found')


if __name__ == '__main__':
    main()

Enter the value of k : 5
Enter name of training data file : iris.csv
Enter name of test data file : iris-test.csv
The predicted class for sample [4.3, 2.9, 1.7, 0.3] is : setosa
Number of votes : 5 out of 5
The predicted class for sample [4.6, 2.7, 1.5, 0.2] is : setosa
Number of votes : 5 out of 5
The predicted class for sample [5.3, 3.4, 1.6, 0.2] is : setosa
Number of votes : 5 out of 5
The predicted class for sample [5.2, 4.1, 1.5, 0.1] is : setosa
Number of votes : 5 out of 5
The predicted class for sample [6.0, 2.2, 4.2, 1.0] is : versicolor
Number of votes : 5 out of 5
The predicted class for sample [6.2, 2.3, 4.5, 1.5] is : versicolor
Number of votes : 4 out of 5
The predicted class for sample [5.0, 2.1, 3.6, 1.2] is : versicolor
Number of votes : 5 out of 5
The predicted class for sample [6.6, 2.8, 5.4, 2.0] is : virginica
Number of votes : 5 out of 5
The predicted class for sample [6.4, 3.2, 5.3, 2.3] is : virginica
Number of votes : 5 out of 5
The predicted class for sample 